In [1]:
import os
import glob
import time
import json
import hashlib
import pandas as pd

from sklearn.model_selection import train_test_split
from catboost import Pool, CatBoostClassifier

Обучаем модель предсказывать события M1 на основе событий M3

In [2]:
# цикл по эксгаустерам
for x in range(4,10):

    y_exh = pd.read_parquet('/root/sep_data/Y_exhauster%s_train.parquet' % x )
    
    cols = y_exh.columns

    # цикл по таргетам
    for target_col in cols:

        y_target = y_exh[[target_col]].copy()
    
        # по таргету в Х оставляем только события М3
        # обнуляем всё, кроме 2
        y_exh[target_col] = (y_target[target_col] == 2)*2
    
        # по таргету в y наоборот, оставляем только 1
        y_target[target_col] = (y_target[target_col] == 1)*1
    
        y = y_exh.copy()
    
        # нам нужен NaN чтобы посчитать кол-во минимум и максимум
        # 0 нам мешает
        for c in y.columns:
            y[c] = y[c]/y[c]*y[c]
    
        # в каждой строке агрегаты за 30 дней
        yy = y.rolling(window='30D').agg(['count','min', 'max'])
    
        # колонки с count делим на кол-во строк в месяце
        for c in yy.columns:
            if c[1] == 'count':
                yy[c] = yy[c]/(30*24*60*6)
    
        features = ["_".join(x) for x in yy.columns]
        
        yy.columns = features
        
        splitted_data = train_test_split(yy, y_target, test_size=0.3)
        X_train, X_test, y_train, y_test = splitted_data
    
        train_pool = Pool(X_train, y_train)
        test_pool = Pool(X_test, y_test)
    
        label_0 = y_target[ y_target[ target_col ] == 0 ].count().values[0]
        label_2 = y_target[ y_target[ target_col ] == 1 ].count().values[0]
        attitude = label_2/(label_0+label_2)
        
        # если очень маленькое отношение таргета к общему кол-ву, то пропускаем
        if attitude < 1e-05:
            print('Skipped target: %s' % target_col, attitude)
            continue
            
        print(target_col)

        # параметры подобрали через  GreedSearchCV
        model = CatBoostClassifier(learning_rate=0.01, iterations=100, depth=5, loss_function='Logloss', eval_metric='Accuracy')
        model.fit(train_pool, eval_set=test_pool, verbose=False)

        model.save_model('/root/m1_model/%s.cb' % hashlib.sha256(target_col.encode('utf-8')).hexdigest() )
        print('Accuracy', model.best_score_.get('learn').get('Accuracy'))
    
    del y_exh


Y_ЭКСГАУСТЕР А/М №4_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №4
Accuracy 0.9999210347870839
Skipped target: Y_ЭКСГАУСТЕР А/М №4_ПОДШИПНИК ОПОРНЫЙ ЭКСГ. №4 0.0
Skipped target: Y_ЭКСГАУСТЕР А/М №4_УЛИТА ЭКСГ. №4 0.0
Y_ЭКСГАУСТЕР А/М №4_РЕДУКТОР ГАЗ. ЗАДВИЖКИ ЭКСГ. №4
Accuracy 0.9999917361986483
Skipped target: Y_ЭКСГАУСТЕР А/М №4_МАСЛОНАСОС РАБОЧИЙ ЭКСГ. №4 0.0
Y_ЭКСГАУСТЕР А/М №4_ЭЛЕКТРОАППАРАТУРА ЭКСГ. №4
Accuracy 0.9999906649651398
Skipped target: Y_ЭКСГАУСТЕР А/М №4_ПОДШИПНИК ОПОРНЫЙ №2 ЭКСГ. №4 5.141920211538597e-06
Y_ЭКСГАУСТЕР А/М №4_ЗАДВИЖКА ЭКСГ. №4
Accuracy 0.9998936418159367
Skipped target: Y_ЭКСГАУСТЕР А/М №4_ГСМ ЭКСГ. №4 0.0
Skipped target: Y_ЭКСГАУСТЕР А/М №4_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№4 Т.1 0.0
Skipped target: Y_ЭКСГАУСТЕР А/М №4_ТСМТ-101-010-50М-400 ТЕРМОПР.ПОДШ.Т.1 0.0
Y_ЭКСГАУСТЕР А/М №4_РОТОР ЭКСГ. №4
Accuracy 0.9997995263005425
Skipped target: Y_ЭКСГАУСТЕР А/М №4_МАСЛООХЛАДИТЕЛЬ  М-05-1 ЭКСГ. №4 0.0
Y_ЭКСГАУСТЕР А/М №4_ПОДШИПНИК ОПОРНО-УПОРНЫЙ ЭКСГ. №4
Accuracy 0.

In [2]:
with open('/root/model/Y_columns.json', 'rb') as f:
    Y_columns = json.loads( f.read().decode('utf-8') ) 

In [3]:
# загружаем наши предсказания М3
y_pred = pd.read_parquet('/root/sep_data/sep_y_pred.parquet')

In [4]:
y_m1 = y_pred[['DT']].copy()
y_m1.index = y_m1['DT']
del y_m1['DT']
y_pred.index = y_pred['DT']
del y_pred['DT']

In [9]:
y_pred_add = y_m1.copy()

In [ ]:
# пропущенные колонки заполняем 0
for col in Y_columns:
    if col in y_pred.columns:
        y_pred_add[ col ] = y_pred[ col ]
    else:
        y_pred_add[ col ] = 0

In [11]:
# подготоавливаем данные
y = y_pred_add
    
for c in y.columns:
    y[c] = y[c]/y[c]*y[c]

yy = y.rolling(window='30D').agg(['count','min', 'max'])

for c in yy.columns:
    if c[1] == 'count':
        yy[c] = yy[c]/(30*24*60*6)

features = ["_".join(x) for x in yy.columns]

yy.columns = features

In [12]:
# диапазаны таргетов к эксгуастерам
target_col_data = {
    "exhauster4": slice(93,116), 
    "exhauster5": slice(28,62),
    "exhauster6": slice(148,175),
    "exhauster7": slice(116,148),
    "exhauster8": slice(62,93),
    "exhauster9": slice(0,28),
}

In [15]:

# цикл по эксгуастерам
for x in target_col_data:
    print(x)

    # цикл по таргетам
    for y in Y_columns[ target_col_data[ x ] ]:
        target_col = y

        # если есть модель, восстанавливаем
        if os.path.exists('/root/m1_model/%s.cb' % hashlib.sha256(target_col.encode('utf-8')).hexdigest() ):
            print(target_col)
            
    
            model = CatBoostClassifier(learning_rate=0.01, iterations=100, depth=5, loss_function='Logloss', eval_metric='Accuracy')
            model = model.load_model('/root/m1_model/%s.cb' % hashlib.sha256(target_col.encode('utf-8')).hexdigest() )
            
            # Делаем предсказание
            y_m1_col_perd = model.predict(yy[model.feature_names_])
            y_m1[target_col] = y_m1_col_perd



exhauster4
Y_ЭКСГАУСТЕР А/М №4_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №4
Y_ЭКСГАУСТЕР А/М №4_РЕДУКТОР ГАЗ. ЗАДВИЖКИ ЭКСГ. №4
Y_ЭКСГАУСТЕР А/М №4_ЭЛЕКТРОАППАРАТУРА ЭКСГ. №4
Y_ЭКСГАУСТЕР А/М №4_ЗАДВИЖКА ЭКСГ. №4
Y_ЭКСГАУСТЕР А/М №4_РОТОР ЭКСГ. №4
Y_ЭКСГАУСТЕР А/М №4_ПОДШИПНИК ОПОРНО-УПОРНЫЙ ЭКСГ. №4
Y_ЭКСГАУСТЕР А/М №4_ЭКСГАУСТЕР А/М №4
exhauster5
Y_ЭКСГАУСТЕР А/М №5_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №5
Y_ЭКСГАУСТЕР А/М №5_ЗАДВИЖКА ЭКСГ. №5
Y_ЭКСГАУСТЕР А/М №5_ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГАУСТЕРА №5
Y_ЭКСГАУСТЕР А/М №5_ПОДШИПНИК ОПОРНЫЙ №1
Y_ЭКСГАУСТЕР А/М №5_ТИРИСТ. ВОЗБУДИТЕЛЬ ВТ-РЭМ-400 ЭКСГ5 ВУ1
Y_ЭКСГАУСТЕР А/М №5_ЭКСГАУСТЕР А/М №5
Y_ЭКСГАУСТЕР А/М №5_РОТОР ЭКСГ. №5
Y_ЭКСГАУСТЕР А/М №5_ГАЗОВАЯ ЗАДВИЖКА ЭКСГАУСТЕРА А/М №5
Y_ЭКСГАУСТЕР А/М №5_ТИРИСТ. ВОЗБУДИТЕЛЬ ВТ-РЭМ-400 ЭКСГ5 ВУ2
exhauster6
Y_ЭКСГАУСТЕР А/М №6_ЗАДВИЖКА ЭКСГ. №6
Y_ЭКСГАУСТЕР А/М №6_ГАЗОВАЯ ЗАДВИЖКА ЭКСГАУСТЕРА А/М №6
Y_ЭКСГАУСТЕР А/М №6_ЭКСГАУСТЕР А/М №6
exhauster7
Y_ЭКСГАУСТЕР А/М №7_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ

In [39]:
# Смотрим, что мы там на предсказывали
# кол-во событий М1 по оборудованию
y_m1.sum().groupby(level=0).sum()

Y_ЭКСГАУСТЕР А/М №4_ЗАДВИЖКА ЭКСГ. №4                           0
Y_ЭКСГАУСТЕР А/М №4_ПОДШИПНИК ОПОРНО-УПОРНЫЙ ЭКСГ. №4           0
Y_ЭКСГАУСТЕР А/М №4_РЕДУКТОР ГАЗ. ЗАДВИЖКИ ЭКСГ. №4             0
Y_ЭКСГАУСТЕР А/М №4_РОТОР ЭКСГ. №4                              0
Y_ЭКСГАУСТЕР А/М №4_ЭКСГАУСТЕР А/М №4                           0
Y_ЭКСГАУСТЕР А/М №4_ЭЛЕКТРОАППАРАТУРА ЭКСГ. №4                  0
Y_ЭКСГАУСТЕР А/М №4_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №4     0
Y_ЭКСГАУСТЕР А/М №5_ГАЗОВАЯ ЗАДВИЖКА ЭКСГАУСТЕРА А/М №5         0
Y_ЭКСГАУСТЕР А/М №5_ЗАДВИЖКА ЭКСГ. №5                           0
Y_ЭКСГАУСТЕР А/М №5_ПОДШИПНИК ОПОРНЫЙ №1                        0
Y_ЭКСГАУСТЕР А/М №5_РОТОР ЭКСГ. №5                              0
Y_ЭКСГАУСТЕР А/М №5_ТИРИСТ. ВОЗБУДИТЕЛЬ ВТ-РЭМ-400 ЭКСГ5 ВУ1    0
Y_ЭКСГАУСТЕР А/М №5_ТИРИСТ. ВОЗБУДИТЕЛЬ ВТ-РЭМ-400 ЭКСГ5 ВУ2    0
Y_ЭКСГАУСТЕР А/М №5_ЭКСГАУСТЕР А/М №5                           0
Y_ЭКСГАУСТЕР А/М №5_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №5     0
Y_ЭКСГАУСТ

Всё по нулям. Не сложно было предвидеть такой исход - очень мало событий М1

Возможно, получилось бы предсказывать события М1 по исходым фичам для поиска событий М3.